## 전처리

In [92]:
import pandas as pd
from ast import literal_eval

movies = pd.read_csv('./data/movies_metadata.csv', nrows=30)[['id', 'title', 'genres', 'vote_average']]
movies['genres'] = movies['genres'].apply(literal_eval).apply(lambda genres : ', '.join([g['name'] for g in genres]))
movies['id'] = movies['id'].astype(int)
movies.head(3)

## Embedding

In [86]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
model = SentenceTransformer("jhgan/ko-sroberta-multitask").to(device=device)

embeddings = model.encode(movies['title'], convert_to_numpy=True, show_progress_bar=True)

* embedding 컬럼 추가

In [87]:
movies['embeddings'] = embeddings.tolist()
movies.head(3)

* dimension 체크

In [88]:
movies['embeddings'].size

## 콜렉션 생성
* Collection : RDB의 Table과 비슷. 하나 이상의 파티션으로 구성. 기본적으로 단일 컬렉션에는 두 개의 샤드가 포함된다.
* DataType 및 속성 : https://milvus.io/docs/create_collection.md

In [89]:
from pymilvus import FieldSchema, CollectionSchema, DataType

def init_schema() -> CollectionSchema :
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=256),
        FieldSchema(name="genres", dtype=DataType.VARCHAR, max_length=256),
        FieldSchema(name="vote_average", dtype=DataType.DOUBLE),
        FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=768) # dim 벡터 차원
    ]
    return CollectionSchema(fields, "movie")

##  Milvus에 데이터 추가

In [90]:
from milvus import default_server
from pymilvus import connections, Collection, utility

with default_server:
    default_server.set_base_dir('milvus_data')

    # 서버 연결
    connections.connect(host='127.0.0.1', port=default_server.listen_port)

    schema = init_schema()
    utility.drop_collection("movie")

    # 컬렉션 생성
    collection = Collection("movie",
                          schema,
                          using="default", # 서버 별칭을 사용하여 컬렉션을 생성할 서버명을 지정 가능
                          shards_num=2 # 샤드 수
                          )

    # 데이터 삽입
    collection.insert(movies)
    collection.flush() # 세그먼트는 특정 크기 이상이어야 sealed 됨. 강제로 sealed하여 인덱싱

    collection.load()
    q = model.encode("money", convert_to_numpy=True, show_progress_bar=True)
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    results = collection.search(q, "embeddings", search_params, limit=3, output_fields=["title"])
    print("test")
    print(results[0][0])